# Orquestación con Prefect

Este notebook documenta y permite ejecutar el **pipeline ETL orquestado** con **Prefect**.  
La lógica del flujo vive en `scripts/etl_fixtures.py` (para producción / CLI); aquí lo consumimos para:
- Mostrar el **setup** (credenciales, rutas, estructura).
- Ejecutar el **flow** de ejemplo.
- Indicar cómo **servir** el flujo y monitorearlo desde **Prefect Cloud**.

> **Nota:** El notebook manual (`ETL_API_Football.ipynb`) NO depende de Prefect.

## 1. Configuración de entorno y rutas

In [3]:
# Este bloque asegura que exista la estructura mínima del datalake
# al correr el flujo desde el notebook (modo demo/documentación).
# Nota: en el script etl_fixtures.py las rutas se definen de nuevo
# para la ejecución real por CLI / Prefect.

import os

DATALAKE_ROOT = "data/etl_datalake"
BRONZE_FIXTURES = f"{DATALAKE_ROOT}/bronze/api_football/fixtures"
SILVER_FIXTURES = f"{DATALAKE_ROOT}/silver/api_football/fixtures"
GOLD_FIXTURES   = f"{DATALAKE_ROOT}/gold/api_football/fixtures"
EXPORTS_DIR     = f"{DATALAKE_ROOT}/exports"

for p in [BRONZE_FIXTURES, SILVER_FIXTURES, GOLD_FIXTURES, EXPORTS_DIR, "scripts"]:
    os.makedirs(p, exist_ok=True)

print("Estructura base creada/validada ✅")

Estructura base creada/validada ✅


## 2. Credenciales y configuración

In [4]:
# Se lee la configuración desde pipeline.conf
# Contiene dos secciones:
# [api-credentials] -> API-Football
# [prefect]         -> Prefect Cloud

from configparser import ConfigParser

parser = ConfigParser()
parser.read("pipeline.conf")

# --- API-Football ---
BASE_URL = parser["api-credentials"]["base_url"]
API_KEY  = parser["api-credentials"]["api_key"]

print("Config OK ✅")
print("BASE_URL:", BASE_URL)
print("API_KEY:  (oculto) ****")

Config OK ✅
BASE_URL: https://v3.football.api-sports.io
API_KEY:  (oculto) ****


## 3. Prefect: conexión y conceptos

Prefect permite **orquestar** un pipeline en pasos claros:

- **Flow** → el contenedor principal del proceso ETL.  
- **Task** → cada paso atómico dentro del flow (ej. extracción, transformación, carga).  
- **Serve / Deploy** → maneras de ejecutar y programar flows:  
  - *Serve*: correrlo como servicio local (útil en desarrollo).  
  - *Deploy*: registrarlo en Prefect Cloud para programarlo y monitorearlo.  

El login en Prefect Cloud se realiza *una sola vez desde la terminal*.
Aquí solo se muestra de manera ilustrativa.

In [5]:
# Prefect requiere login en Cloud, pero este paso se hace normalmente
# desde la terminal (una única vez por entorno).
# Aquí solo mostramos cómo se lee la API Key desde pipeline.conf.

prefect_credentials = parser["prefect"]
prefect_api_key = prefect_credentials["api_key"]

print("Prefect API Key cargada (oculta) ****")

# Login en Prefect Cloud (solo a modo ilustrativo; en práctica se hace una vez desde la terminal)
# !prefect cloud login -k {prefect_api_key}

Prefect API Key cargada (oculta) ****


In [6]:
import prefect
print("Prefect versión:", prefect.__version__)

Prefect versión: 2.20.9


## 4. Uso del flujo desde `scripts/etl_fixtures.py`

El flujo ETL está definido en `scripts/etl_fixtures.py`.  
En este notebook es posible:

- **4.1 Opción A — Corrida única (demo one-off):** ejecuta el flow una sola vez para validar la orquestación.  
- **4.2 Opción B — Servir el flow (opcional):** mantiene el flow activo como servicio local.

### 4.1 Opción A — Ejecutar una corrida orquestada (demo one-off)

In [2]:
import importlib

# Importa el script de orquestación
etl = importlib.import_module("scripts.etl_fixtures")

# Corre el flow una vez, de forma local
etl.etl_parametrizable(endpoints=["fixtures"])

17:43:59.512 | INFO    | prefect.engine - Created flow run 'saffron-lobster' for flow 'etl-parametrizable'

17:43:59.512 | INFO    | Flow run 'saffron-lobster' - View at https://app.prefect.cloud/account/1513bf29-3686-40b8-9dbf-c85ba6a6f8c0/workspace/377710aa-6343-48a1-a52d-8fd9be6fbba7/flow-runs/flow-run/068d5a98-f895-7158-8000-5870901a27a8

17:44:00.210 | INFO    | Flow run 'saffron-lobster' - Created task run 'extract_data-0' for task 'extract_data'

17:44:00.210 | INFO    | Flow run 'saffron-lobster' - Executing 'extract_data-0' immediately...

17:44:02.658 | INFO    | Task run 'get_data-fixtures' - Finished in state Completed()

...

17:44:03.280 | INFO    | Flow run 'saffron-lobster' - Executing 'transform_data-0' immediately...

17:44:04.493 | INFO    | Task run 'transform_data' - Finished in state Completed()

17:44:04.692 | ERROR   | Flow run 'saffron-lobster' - Encountered exception during execution:
Traceback (most recent call last):
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\prefect\engine.py", line 894, in orchestrate_flow_run
    result = await flow_call.aresult()
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\prefect\_internal\concurrency\calls.py", line 327, in aresult
    return await asyncio.wrap_future(self.future)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\anaconda3\envs\etl-football\Lib\site-packages\prefect\_internal\concurrency\calls.py", line 352, in _run_sync
    result = self.fn(*self.args, **self.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elias\OneDrive\Documentos\Repositorios_notebooks\ETL_api_football\scripts\etl_fixtures.py", line 167, in etl_parametrizable
    load_data(df_bronze, endpoint)
    ^^^^^^^^^
NameError: name 'load_data' is not defined

17:44:04.961 | ERROR   | Flow run 'saffron-lobster' - Finished in state Failed("Flow run encountered an exception. NameError: name 'load_data' is not defined")

NameError: name 'load_data' is not defined

### 4.2 Opción B — Servir el flow como agente local (opcional)  
Esta modalidad deja el flujo en ejecución continua y permite gestionarlo desde Prefect Cloud (ejecuciones, logs, programación).  
⚠️ La celda quedará ocupada hasta que la detengas manualmente desde tu entorno (Interrupt/Stop del notebook).
Por defecto está comentada para evitar que se ejecute por accidente.

In [ ]:
import importlib
etl = importlib.import_module("scripts.etl_fixtures")
# etl.etl_parametrizable.serve(name="ETL-Fixtures", endpoints=["fixtures"])

## 5. Monitoreo en Prefect

El monitoreo del flujo no se realiza desde el notebook, sino desde la **UI de Prefect Cloud**.  
Una vez que el flow se ejecuta (corrida única o servido como agente), es posible:

1. **Flows** → ver el listado de flows registrados (ejemplo: `ETL-Fixtures`).  
2. **Run history** → revisar el historial de ejecuciones con sus logs, tiempos y reintentos.  
3. **Blocks** → administrar la configuración de almacenamiento o infraestructura remota si se utiliza.  
4. **Schedules** → programar ejecuciones automáticas (requiere plan pago).